In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt 
import numpy as np 
from visibility_graph import visibility_graph
import networkx as nx
import statsmodels.api as sm
import os

ruta = '../LPV/'
title = 'LPV'

edge = '../LPV/edgeList/'

if not os.path.exists(ruta):
    os.makedirs(ruta)

if not os.path.exists(edge):
    os.makedirs(edge)

In [2]:
def lista0 (lista1, lista2):
    while 0 in lista1:
        lista2.pop(lista1.index(0))
        lista1.remove(0)
    return lista1, lista2 

def get_alpha(route,id, li_fit, ls_fit, xlimi, xlims, color, name):
    
    data = f'{route}{id}'
    routeSaved = f'{route}pdf_{title}/'

    if not os.path.exists(routeSaved):
        os.makedirs(routeSaved)

    #manejo del grafo y grados.
    G = nx.read_edgelist(data,nodetype=int) # leemos el grafo
    degree_count =  nx.degree_histogram(G) 
    degrees = list(range(0, len(degree_count)))

    degree_count, degrees = lista0(degree_count, degrees)
    
    #Normalize the degree distribution
    degree_distribution = [count/float(sum(degree_count)) for count in degree_count]
    x0,y0=np.array(np.log10(degrees)),np.array(np.log10(degree_distribution))
    x,y=x0[(x0>=li_fit)&(x0<=ls_fit)],y0[(x0>=li_fit)&(x0<=ls_fit)]


    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()

    # Plot degree distribution
    a=np.linspace(li_fit,ls_fit,10)
    

    plt.figure(figsize=(6,4))
    plt.plot(x0,y0,color=color,linewidth=0,marker="P",markersize=5,label="data")
    plt.plot(a,(a)*(model.params[1])+model.params[0]*1.,color="k",lw=3,label=r"fit ($\alpha_0={}$)".format(-np.round(model.params[1],2)))
    plt.xlabel(r'$\log_{10}(k)$ (Degree)'); plt.ylabel(r'$\log_{10} P(k)$'); plt.title('Degree Distribution {}'.format(name)); plt.legend(); plt.xlim(xlimi,xlims)
    plt.legend(title=r"$P(k)\sim x^{-\alpha_0}$"); plt.grid(alpha=0.5)
    plt.savefig(routeSaved+f'{id}.pdf',dpi=400,bbox_inches='tight')
    plt.show()

    return

In [3]:
lpv = pd.read_csv(f'../transients/{title}.csv')
lpv

,Classification,ID,observation_id,Mag,Magerr,MJD
0,LPV,1112291120524129371,151479,13.8415,0.054039,54548.315422
1,LPV,1206181040794128810,284577,15.1438,0.059172,55625.497183
2,LPV,1206181040794128810,284578,15.2550,0.059927,55634.424365
3,LPV,1206181040794128810,284553,15.8084,0.061762,55656.399305
4,LPV,1206181040794128810,284552,15.7952,0.061654,55656.393029
...,...,...,...,...,...,...
1834,LPV,1202251040804151905,284694,19.0625,0.178905,53509.182677
1835,LPV,1202251040804151905,284693,20.2789,0.338933,53509.176253
1836,LPV,1202251040804151905,284692,19.6784,0.246706,53494.384739
1837,LPV,1202251040804151905,284696,17.8148,0.118167,53834.335148


In [12]:
ids = list(lpv['ID'].unique())
print(len(list(lpv['ID'].unique())))
lpv['ID'].unique()


5


array([1112291120524129371, 1206181040794128810, 1412301211134147029,
       1202251040804151905, 1302141290364160994], dtype=int64)

##### Función para hacer los datos

In [5]:
#funcion que haga eso de forma automatica

def edgelist(id):

    vec_id = lpv[lpv['ID'] == id]['Mag']
    graph_id = visibility_graph(vec_id)
    nx.write_edgelist(graph_id, f'{ruta}edgeList/{id}')


#### Indices de los id's a probar (sacado aleatoreamente)

solo hay 5 elementos, así que va en orden xd


In [6]:
id1 = 1112291120524129371
id2 = 1206181040794128810
id3 = 1412301211134147029
id4 = 1202251040804151905
id5 = 1302141290364160994


edgelist(id1)
edgelist(id2)
edgelist(id3)
edgelist(id4)
edgelist(id5)

In [5]:
def get_alpha_data(route,id, li_fit, ls_fit, name):
    
    data = f'{edge}{id}'
    
    #manejo del grafo y grados.
    G = nx.read_edgelist(data,nodetype=int) # leemos el grafo
    degree_count =  nx.degree_histogram(G) 
    degrees = list(range(0, len(degree_count)))

    degree_count, degrees = lista0(degree_count, degrees)
    
    #Normalize the degree distribution
    degree_distribution = [count/float(sum(degree_count)) for count in degree_count]
    x0,y0=np.array(np.log10(degrees)),np.array(np.log10(degree_distribution))
    x,y=x0[(x0>=li_fit)&(x0<=ls_fit)],y0[(x0>=li_fit)&(x0<=ls_fit)]


    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()

    alpha = -np.round(model.params[1],2)
    values = [name,id,alpha]
    
    return values

In [8]:
values = []
for id in ids:
    values.append(get_alpha_data(edge, id, 0.766, 1.447, 'LPV')) 

print(values)

[['LPV', 1112291120524129371, 2.52], ['LPV', 1206181040794128810, 2.39], ['LPV', 1412301211134147029, 2.57], ['LPV', 1202251040804151905, 2.27], ['LPV', 1302141290364160994, 2.46]]


In [9]:
import csv 

ruta = '../resultados/prueba.csv'

with open (ruta, mode = 'a', newline = '') as archivo: 
    writer = csv.writer(archivo)

    writer.writerows(values)
    pass

In [7]:
values_min = [0.72, 0.780, 0.70, 0.90, 0.73]
values_max = [1.50, 1.435, 1.39, 1.49, 1.42]
values_alpha = [2.35, 2.39, 2.6, 2.09, 2.47]

#### PRUEBA 1 $\rightarrow$ LPV

ID: 1112291120524129371

In [8]:
#0.85 & 1.34 -> 3.0
#otra opcion: 0.65  & 1.43 -> 2.32 [posible ganadora]
#otra (no creo tanto): 0.72 & 1.42 -> 2.4

# get_alpha(ruta, id1, 0.72,1.5,0.26,1.78, "red", id1)


#### PRUEBA 2 $\rightarrow$ LPV

ID: 1206181040794128810

In [9]:
#0.85 & 1.34 -> 1.58
#otra opcion: 0.65  & 1.398 -> 2.19 [posible ganadora]
#otra: 0.695 & 1.398 -> 2.19 [mismo pero mejor]

# get_alpha(ruta,id2,0.78,1.435,0.3,1.75, "red", id2)


#### PRUEBA 3 $\rightarrow$ LPV

ID: 1412301211134147029

In [10]:


# get_alpha(ruta,id3,0.70,1.39,0.3,1.75, "red", id3)


#### PRUEBA 4 $\rightarrow$ LPV

ID: 1202251040804151905

In [11]:

#dudo un poco 

# get_alpha(ruta,id4,0.90,1.49,0.3,1.75, "red", id4)


#### PRUEBA 5 $\rightarrow$ LPV

ID: 1302141290364160994

In [12]:


# get_alpha(ruta,id5,0.73,1.42,0.3,1.75, "red", id5)
